### Imports

In [1]:
import tensorflow as tf
import numpy as np
import re
#import itertools
#from collections import Counter
import pandas as pd
from tensorflow.contrib import learn
import os
import time
import datetime

### Parameters

In [2]:
# improvements
# L2, dropout, CV, more layers, TF records
# remove stuff that isn't being used
# word embeddings... 
# data cleaning...
# cleaner way to tun parameters...
# talk about algors: CNNs, LSTMs, maybe fasttext
# conditional autostop training...
# check/prevent over fitting...
# tensorboard(launch???)
### dopeee i can watch training in real time with tensorboard!!!
### 1515401822, loss curve shows location of over fit... 4200 step lowest loss before incline
# post to github
# update python code so it works with local file structure
# remove flags

In [3]:
## Can only run this block of code once due to flags...
## better to use global variables vs FLAGS??
# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("spooky_data_file", "./data/spooky_data/spooky.data.csv", "Data source for the spooky data.")
tf.flags.DEFINE_string("spooky_test_file", "./data/spooky_data/spooky.test.csv", "Data source for the spooky data.")


# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 100, "Batch Size (default: 64)")
# 25 overshoots with batch 64, 
tf.flags.DEFINE_integer("num_epochs", 20 , "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

# Eval Parameters
tf.flags.DEFINE_string("checkpoint_dir", "", "Checkpoint directory from training run")
tf.flags.DEFINE_boolean("eval_train", False, "Evaluate on all training data")
tf.flags.DEFINE_boolean("eval_test", True, "Evaluate on all training data")

### Data Preparation

#### Data Functions

In [4]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def load_test_data(spooky_test_file):
    # reallllllly un organized
    # Load data from files
    spooky_data = pd.read_csv(spooky_test_file, skiprows=1, 
                              names=['id', 'text'])

    test_examples = spooky_data['text']

    datasets = dict()
    datasets['data'] = test_examples

    x_text = datasets['data']
    x_text = [clean_str(sent) for sent in x_text]
    #x_text = [clean_stop(sent) for sent in x_text]
    # ids, odd transformations from dfs and dicts
    ids = dict()
    x_ids = spooky_data['id']
    ids['data'] = x_ids

    x_ids = ids['data']
    return [x_text, x_ids]

def get_datasets_spooky(spooky_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    spooky_data = pd.read_csv(spooky_data_file, skiprows=1, 
                              names=['id', 'text', 'author']).set_index('id')
    # double stripping???
    eap_examples = spooky_data[spooky_data['author'] == 'EAP']['text']
    eap_examples = [s.strip() for s in eap_examples]
    hpl_examples = spooky_data[spooky_data['author'] == 'HPL']['text']
    hpl_examples = [s.strip() for s in hpl_examples]
    mws_examples = spooky_data[spooky_data['author'] == 'MWS']['text']
    mws_examples = [s.strip() for s in mws_examples]

    datasets = dict()
    datasets['data'] = eap_examples + hpl_examples + mws_examples
    target = [0 for x in eap_examples] + [1 for x in hpl_examples] + [2 for x in mws_examples]
    datasets['target'] = target
    datasets['target_names'] = ['eap_examples', 'hpl_examples', 'mws_examples']
    return datasets


def load_data_labels(datasets):
    """
    Load data and labels
    :param datasets:
    :return:
    """
    # Split by words
    x_text = datasets['data']
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    labels = []
    for i in range(len(x_text)):
        label = [0 for j in datasets['target_names']]
        label[datasets['target'][i]] = 1
        labels.append(label)
    y = np.array(labels)
    return [x_text, y]


def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
#==================================================
# added for probabilities
#==================================================
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    if x.ndim == 1:
        x = x.reshape((1, -1))
    max_x = np.max(x, axis=1).reshape((-1, 1))
    exp_x = np.exp(x - max_x)
    return exp_x / np.sum(exp_x, axis=1).reshape((-1, 1))

#### Data Preparation

In [5]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

print("\nEvaluating...\n")

# CHANGE THIS: Load data. Load your own data here
if FLAGS.eval_test:
    x_raw, x_ids = load_test_data(FLAGS.spooky_test_file)
    print("Total number of test examples: {}".format(len(x_raw)))
else:
    x_raw = ["i wish i were", "an oscar meyer"]

# Load data
print("Loading data...")
# x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)
datasets = get_datasets_spooky(FLAGS.spooky_data_file)
x_text, y = load_data_labels(datasets)

# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

del x, y, x_shuffled, y_shuffled

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=100
CHECKPOINT_DIR=
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVAL_TEST=True
EVAL_TRAIN=False
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NUM_CHECKPOINTS=5
NUM_EPOCHS=20
NUM_FILTERS=128
SPOOKY_DATA_FILE=./data/spooky_data/spooky.data.csv
SPOOKY_TEST_FILE=./data/spooky_data/spooky.test.csv


Evaluating...

Total number of test examples: 8392
Loading data...
Vocabulary Size: 25428
Train/Dev split: 17622/1957


### Model

In [6]:
class TextDNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)

        self.h_pool = tf.sqrt(tf.reduce_sum(tf.square(self.embedded_chars), 1))

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            self.scores = tf.nn.xw_plus_b(self.h_pool, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            preds = tf.sigmoid(self.scores)
            self.loss = tf.losses.log_loss(labels=self.input_y, predictions=preds)

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

### Training

In [7]:
# reset function if not below 1 before 50
# maybe just run 5 times per parameter change
# Training
# ============================
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextDNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
                
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            # add print epoch
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name W:0/grad/hist is illegal; using W_0/grad/hist instead.
INFO:tensorflow:Summary name W:0/grad/sparsity is illegal; using W_0/grad/sparsity instead.
INFO:tensorflow:Summary name output/b:0/grad/hist is illegal; using output/b_0/grad/hist instead.
INFO:tensorflow:Summary name output/b:0/grad/sparsity is illegal; using output/b_0/grad/sparsity instead.
Writing to /home/hi/Documents/Nanodegree/machine-learning-master/cnn-text-classification-tf_OG/runs/1515404745

2018-01-08T04:45:46.181954: step 1, loss 5.69721, acc 0.25
2018-01-08T04:45:46.319815: step 2, loss 4.04224, acc 0.35
2018-01-08T04:45:46.462743: step 3, loss 3.45145, acc 0.36
2018-01-08T04:45:46.608299: step 4, loss 3.30845, acc 0.28
2018-01-08T04:45:46.746337: step 5, loss 2.51854, acc 0.

2018-01-08T04:46:03.935035: step 122, loss 0.632398, acc 0.42
2018-01-08T04:46:04.074904: step 123, loss 0.622931, acc 0.39
2018-01-08T04:46:04.214136: step 124, loss 0.602859, acc 0.47
2018-01-08T04:46:04.353707: step 125, loss 0.594796, acc 0.57
2018-01-08T04:46:04.495768: step 126, loss 0.612372, acc 0.42
2018-01-08T04:46:04.642213: step 127, loss 0.5863, acc 0.53
2018-01-08T04:46:04.782787: step 128, loss 0.610025, acc 0.43
2018-01-08T04:46:04.920514: step 129, loss 0.608125, acc 0.4
2018-01-08T04:46:05.061228: step 130, loss 0.617853, acc 0.39
2018-01-08T04:46:05.197514: step 131, loss 0.609605, acc 0.53
2018-01-08T04:46:05.339490: step 132, loss 0.603268, acc 0.58
2018-01-08T04:46:05.476678: step 133, loss 0.6044, acc 0.51
2018-01-08T04:46:05.619077: step 134, loss 0.613717, acc 0.49
2018-01-08T04:46:05.764228: step 135, loss 0.605786, acc 0.45
2018-01-08T04:46:05.909191: step 136, loss 0.607665, acc 0.49
2018-01-08T04:46:06.054723: step 137, loss 0.600324, acc 0.42
2018-01-08T04

2018-01-08T04:46:22.767441: step 251, loss 0.560789, acc 0.59
2018-01-08T04:46:22.912142: step 252, loss 0.575114, acc 0.55
2018-01-08T04:46:23.057435: step 253, loss 0.578513, acc 0.55
2018-01-08T04:46:23.200711: step 254, loss 0.551244, acc 0.61
2018-01-08T04:46:23.349969: step 255, loss 0.552101, acc 0.61
2018-01-08T04:46:23.491577: step 256, loss 0.590507, acc 0.5
2018-01-08T04:46:23.642796: step 257, loss 0.572668, acc 0.61
2018-01-08T04:46:23.788714: step 258, loss 0.587411, acc 0.53
2018-01-08T04:46:23.938556: step 259, loss 0.579235, acc 0.61
2018-01-08T04:46:24.086053: step 260, loss 0.556055, acc 0.65
2018-01-08T04:46:24.235319: step 261, loss 0.558251, acc 0.5
2018-01-08T04:46:24.385869: step 262, loss 0.608281, acc 0.41
2018-01-08T04:46:24.538797: step 263, loss 0.605321, acc 0.41
2018-01-08T04:46:24.683963: step 264, loss 0.577528, acc 0.46
2018-01-08T04:46:24.831555: step 265, loss 0.566051, acc 0.6
2018-01-08T04:46:24.979331: step 266, loss 0.575111, acc 0.6
2018-01-08T0

2018-01-08T04:46:43.561390: step 380, loss 0.503616, acc 0.8
2018-01-08T04:46:43.707624: step 381, loss 0.524239, acc 0.65
2018-01-08T04:46:43.852118: step 382, loss 0.525132, acc 0.51
2018-01-08T04:46:43.999737: step 383, loss 0.586978, acc 0.44
2018-01-08T04:46:44.147179: step 384, loss 0.52007, acc 0.57
2018-01-08T04:46:44.292687: step 385, loss 0.527412, acc 0.67
2018-01-08T04:46:44.446764: step 386, loss 0.527858, acc 0.67
2018-01-08T04:46:44.590572: step 387, loss 0.519164, acc 0.77
2018-01-08T04:46:44.741969: step 388, loss 0.519517, acc 0.8
2018-01-08T04:46:44.893596: step 389, loss 0.528122, acc 0.66
2018-01-08T04:46:45.043983: step 390, loss 0.524354, acc 0.62
2018-01-08T04:46:45.190007: step 391, loss 0.507787, acc 0.76
2018-01-08T04:46:45.339199: step 392, loss 0.540341, acc 0.63
2018-01-08T04:46:45.478157: step 393, loss 0.518898, acc 0.71
2018-01-08T04:46:45.617566: step 394, loss 0.534054, acc 0.63
2018-01-08T04:46:45.762099: step 395, loss 0.525567, acc 0.71
2018-01-08T

2018-01-08T04:47:02.451942: step 506, loss 0.499791, acc 0.58
2018-01-08T04:47:02.593502: step 507, loss 0.48927, acc 0.66
2018-01-08T04:47:02.734753: step 508, loss 0.495199, acc 0.78
2018-01-08T04:47:02.873227: step 509, loss 0.506723, acc 0.72
2018-01-08T04:47:03.014942: step 510, loss 0.515711, acc 0.76
2018-01-08T04:47:03.164246: step 511, loss 0.501933, acc 0.76
2018-01-08T04:47:03.300970: step 512, loss 0.493809, acc 0.73
2018-01-08T04:47:03.436861: step 513, loss 0.521367, acc 0.56
2018-01-08T04:47:03.582372: step 514, loss 0.499555, acc 0.61
2018-01-08T04:47:03.727855: step 515, loss 0.508694, acc 0.61
2018-01-08T04:47:03.867769: step 516, loss 0.508263, acc 0.63
2018-01-08T04:47:04.013115: step 517, loss 0.521915, acc 0.7
2018-01-08T04:47:04.159015: step 518, loss 0.49776, acc 0.74
2018-01-08T04:47:04.302243: step 519, loss 0.504728, acc 0.77
2018-01-08T04:47:04.443133: step 520, loss 0.481818, acc 0.81
2018-01-08T04:47:04.590306: step 521, loss 0.508093, acc 0.64
2018-01-08T

...

2018-01-08T04:53:21.866779: step 2917, loss 0.124444, acc 0.96
2018-01-08T04:53:22.003262: step 2918, loss 0.130993, acc 0.98
2018-01-08T04:53:22.149639: step 2919, loss 0.0951692, acc 0.98
2018-01-08T04:53:22.285991: step 2920, loss 0.133491, acc 0.96
2018-01-08T04:53:22.426591: step 2921, loss 0.111922, acc 0.96
2018-01-08T04:53:22.563981: step 2922, loss 0.159619, acc 0.94
2018-01-08T04:53:22.701971: step 2923, loss 0.125077, acc 0.96
2018-01-08T04:53:22.846146: step 2924, loss 0.14864, acc 0.95
2018-01-08T04:53:22.992949: step 2925, loss 0.127244, acc 0.96
2018-01-08T04:53:23.140153: step 2926, loss 0.133699, acc 0.96
2018-01-08T04:53:23.281482: step 2927, loss 0.143719, acc 0.97
2018-01-08T04:53:23.418013: step 2928, loss 0.131074, acc 0.92
2018-01-08T04:53:23.554777: step 2929, loss 0.115394, acc 0.97
2018-01-08T04:53:23.702403: step 2930, loss 0.144266, acc 0.94
2018-01-08T04:53:23.846937: step 2931, loss 0.132989, acc 0.96
2018-01-08T04:53:23.986272: step 2932, loss 0.136619, a

2018-01-08T04:53:42.887911: step 3045, loss 0.114293, acc 0.94
2018-01-08T04:53:43.056594: step 3046, loss 0.116712, acc 0.96
2018-01-08T04:53:43.234730: step 3047, loss 0.115272, acc 0.97
2018-01-08T04:53:43.408079: step 3048, loss 0.105711, acc 0.98
2018-01-08T04:53:43.578393: step 3049, loss 0.122217, acc 0.96
2018-01-08T04:53:43.754251: step 3050, loss 0.119276, acc 0.95
2018-01-08T04:53:43.920685: step 3051, loss 0.114749, acc 0.95
2018-01-08T04:53:44.093458: step 3052, loss 0.11961, acc 0.96
2018-01-08T04:53:44.289014: step 3053, loss 0.0868365, acc 1
2018-01-08T04:53:44.456847: step 3054, loss 0.1547, acc 0.94
2018-01-08T04:53:44.657767: step 3055, loss 0.0857293, acc 0.99
2018-01-08T04:53:44.830042: step 3056, loss 0.155266, acc 0.95
2018-01-08T04:53:45.002534: step 3057, loss 0.116933, acc 0.95
2018-01-08T04:53:45.173427: step 3058, loss 0.117706, acc 0.97
2018-01-08T04:53:45.345084: step 3059, loss 0.136007, acc 0.94
2018-01-08T04:53:45.519832: step 3060, loss 0.103581, acc 0

2018-01-08T04:54:04.664707: step 3173, loss 0.121328, acc 0.95
2018-01-08T04:54:04.833590: step 3174, loss 0.0949892, acc 0.99
2018-01-08T04:54:04.996691: step 3175, loss 0.124568, acc 0.95
2018-01-08T04:54:05.162274: step 3176, loss 0.120018, acc 0.95
2018-01-08T04:54:05.320362: step 3177, loss 0.19264, acc 0.91
2018-01-08T04:54:05.473140: step 3178, loss 0.119965, acc 0.98
2018-01-08T04:54:05.636815: step 3179, loss 0.116155, acc 0.95
2018-01-08T04:54:05.798063: step 3180, loss 0.124529, acc 0.95
2018-01-08T04:54:05.947136: step 3181, loss 0.138081, acc 0.95
2018-01-08T04:54:06.095530: step 3182, loss 0.128935, acc 0.96
2018-01-08T04:54:06.238302: step 3183, loss 0.0719659, acc 0.99
2018-01-08T04:54:06.384636: step 3184, loss 0.177835, acc 0.9
2018-01-08T04:54:06.535113: step 3185, loss 0.0861314, acc 0.99
2018-01-08T04:54:06.624150: step 3186, loss 0.166008, acc 0.909091
2018-01-08T04:54:06.772463: step 3187, loss 0.118948, acc 0.96
2018-01-08T04:54:06.913283: step 3188, loss 0.0999

2018-01-08T04:54:25.259816: step 3300, loss 0.108708, acc 0.97

Evaluation:
2018-01-08T04:54:25.573625: step 3300, loss 0.2425, acc 0.859479

Saved model checkpoint to /home/hi/Documents/Nanodegree/machine-learning-master/cnn-text-classification-tf_OG/runs/1515404745/checkpoints/model-3300

2018-01-08T04:54:25.805311: step 3301, loss 0.12891, acc 0.97
2018-01-08T04:54:25.956668: step 3302, loss 0.10014, acc 0.99
2018-01-08T04:54:26.114081: step 3303, loss 0.135915, acc 0.94
2018-01-08T04:54:26.257906: step 3304, loss 0.141377, acc 0.94
2018-01-08T04:54:26.401682: step 3305, loss 0.116475, acc 0.96
2018-01-08T04:54:26.551433: step 3306, loss 0.123084, acc 0.94
2018-01-08T04:54:26.697531: step 3307, loss 0.107172, acc 0.98
2018-01-08T04:54:26.845029: step 3308, loss 0.134796, acc 0.95
2018-01-08T04:54:26.990103: step 3309, loss 0.112542, acc 0.96
2018-01-08T04:54:27.139751: step 3310, loss 0.107941, acc 0.96
2018-01-08T04:54:27.288708: step 3311, loss 0.0720159, acc 1
2018-01-08T04:54:27

2018-01-08T04:54:45.595816: step 3423, loss 0.1265, acc 0.97
2018-01-08T04:54:45.754016: step 3424, loss 0.122018, acc 0.95
2018-01-08T04:54:45.914178: step 3425, loss 0.0859995, acc 0.96
2018-01-08T04:54:46.062896: step 3426, loss 0.0820111, acc 0.97
2018-01-08T04:54:46.213379: step 3427, loss 0.0770813, acc 0.99
2018-01-08T04:54:46.360369: step 3428, loss 0.107751, acc 0.95
2018-01-08T04:54:46.509881: step 3429, loss 0.108133, acc 0.97
2018-01-08T04:54:46.659472: step 3430, loss 0.145213, acc 0.93
2018-01-08T04:54:46.812021: step 3431, loss 0.0897287, acc 0.96
2018-01-08T04:54:46.963381: step 3432, loss 0.0760395, acc 1
2018-01-08T04:54:47.109608: step 3433, loss 0.0928793, acc 0.97
2018-01-08T04:54:47.256482: step 3434, loss 0.0942939, acc 0.98
2018-01-08T04:54:47.398079: step 3435, loss 0.082912, acc 0.98
2018-01-08T04:54:47.545632: step 3436, loss 0.100665, acc 0.98
2018-01-08T04:54:47.693403: step 3437, loss 0.120956, acc 0.95
2018-01-08T04:54:47.843253: step 3438, loss 0.11144, 

### Evaluation

In [9]:
# Map data into vocabulary
#print(FLAGS.checkpoint_dir)
#vocab_path = os.path.join(FLAGS.checkpoint_dir, "..", "vocab")
#print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
#print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
#vocab_path = os.path.join(FLAGS.checkpoint_dir, "..", "vocab")
vocab_path = os.path.join(checkpoint_dir, "..", "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
x_test = np.array(list(vocab_processor.transform(x_raw)))

# Evaluation
# ==================================================
#checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)

graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        #### Tensors we want to evaluate
        scores = graph.get_operation_by_name("output/scores").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []
        all_probabilities = None

        for x_test_batch in batches:
            batch_predictions_scores = sess.run([predictions,scores], {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions_scores[0]])
            probabilities = softmax(batch_predictions_scores[1])
            if all_probabilities is not None:
                all_probabilities = np.concatenate([all_probabilities, probabilities])
            else:
                all_probabilities = probabilities


import pandas as pd
df_ids = pd.DataFrame(x_ids, columns = ['id'])
df_probs = pd.DataFrame(all_probabilities, columns = ['EAP','HPL','MWS'])
df_sub = pd.concat([df_ids,df_probs], axis =1, join='inner')
out_path = os.path.join(FLAGS.checkpoint_dir, "", "submission.csv")
print("Saving evaluation to {0}".format(out_path))
df_sub.to_csv(out_path, index = False)
print(df_sub.head())

INFO:tensorflow:Restoring parameters from /home/hi/Documents/Nanodegree/machine-learning-master/cnn-text-classification-tf_OG/runs/1515404745/checkpoints/model-3500
Saving evaluation to submission.csv
        id       EAP       HPL       MWS
0  id02310  0.008586  0.000546  0.990868
1  id24541  0.999425  0.000572  0.000004
2  id00134  0.029484  0.970295  0.000221
3  id27757  0.992481  0.006712  0.000807
4  id04081  0.961886  0.005635  0.032479
